# distance fitness functions

In `distance.py`, there is an implementation of distance search.
Here, let us try use it!


`distGenetic`function in `distance.py` has input variables, which

- `L` (numpy.ndarray): Logical Pauli basis in binary matrix form.
        Each row represents one logical operator.
        Shape: (k, n) where k is the number of logical operators and n is the code length.
-  `S` (numpy.ndarray, optional): Stabilizer generators in binary matrix form.
        Each row represents one stabilizer operator.
        Shape: (r, n) where r is the number of stabilizer generators.
        Defaults to None for classical codes.
- `tB` (int, optional): Number of blocks.
        Set to 1 for CSS or classical codes.
        Set to 2 for non-CSS codes.
        Defaults to 1.
-  `pList` (list, optional): List of probabilities for X, Y, and Z errors.
        Set to None for depolarizing noise.
        Defaults to None.
- `settgs` (dict, optional): Dictionary of algorithm settings.(explained a later section)


First, let define 5-qubit code
logical operators are


In [1]:
import numpy as np

from add_parent_dir import *
from distance import distGenetic, distGeneticEvalInterface

In [2]:
L = np.array([
    [1,1,1,1,1, 0,0,0,0,0],
    [0,0,0,0,0, 1,1,1,1,1]
])

L

array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])

Stabilizer generators are

In [3]:
S = np.array([
    [1,0,0,1,0, 0,1,1,0,0],
    [0,1,0,0,1, 0,0,1,1,0],
    [1,0,1,0,0, 0,0,0,1,1],
    [0,1,0,1,0, 1,0,0,0,1]
])

S

array([[1, 0, 0, 1, 0, 0, 1, 1, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 0, 0, 0, 1]])

Here, we'd like to treat non-CSS stabilizers.
So let us set `tB=2`.

plist is supposed to be depolarizing type noise `[0.001, 0.001, 0.001]`

Settings requires parameters blow. 

- **`mu`**: Total population for each generation
- **`lambmu`**: Ratio of offspring for each parent lambda to the total population mu
- **`pMut`**: Mutation probability
- **`sMut`**: Mutation standard deviation
- **`genCount`**: Number of generations
- **`tabuLength`**: A tabu list of the most recently explored individuals is maintained, and these are not revisited. `tabuLength` defines the size of this list
- **`fast`**: Setting to make the algorithm run quickly by modifying the number of generations, population etc.

Default Values

```python
defs = {
    'lambmu': 3,
    'mu': nB,
    'pMut': 2 / nB,
    'sMut': 1 / (nB * nB),
    'genCount': 1 + int(np.log(nB)),
    'tabuLength': nB,
    'fast': True
}
```
where `nB` represents the number of blocks (`n // tB`).

To use this function as DistRand, we set `genCount=1` via advice from Mark Webstar.

In [4]:
tB = 2
ratio = [1,1,1]
probability = 0.001
plist = np.array([1-probability] + [probability/3] * (2**tB-1))

settings = {
    'mu': 1,
    'lambmu': 1,
    'pMut': 0,
    'sMut': 0,
    'genCount': 0,
    'fast': True
}


Run the function

In [5]:
probList, LOList, population = distGeneticEvalInterface(L, S=S, tB=tB, pList=plist, settgs=settings)

print(
    "probList",
    probList
)
print(
    "LOList",
    LOList
)
print(
    "population",
    population
)


[[1 1 0 0 1 1 0 0 0 0 1 1]
 [1 0 0 0 0 0 1 0 0 1 1 0]] [3.6963e-11 3.6963e-11]
probList [array([3.6963e-11, 3.6963e-11])]
LOList [array([[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]])]
population [array([1, 3, 2, 0, 4])]


In [23]:
distGenetic(L, S=S, tB=tB, pList=plist, settgs=settings)

(np.float64(1.9960020000000004e-09),
 array([[0, 1, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 1, 0, 1, 0]]))

## Experiment using Steane code

| Generator | q1 | q2 | q3 | q4 | q5 | q6 | q7 |
|-----------|----|----|----|----|----|----|----| 
| g1        | I  | I  | I  | X  | X  | X  | X  |
| g2        | I  | X  | X  | I  | I  | X  | X  |
| g3        | X  | I  | X  | I  | X  | I  | X  |
| g4        | I  | I  | I  | Z  | Z  | Z  | Z  |
| g5        | I  | Z  | Z  | I  | I  | Z  | Z  |
| g6        | Z  | I  | Z  | I  | Z  | I  | Z  | 
| L1        | X  | X  | X  | X  | X  | X  | X  |
| L2        | Z  | Z  | Z  | Z  | Z  | Z  | Z  |



In [37]:
L_steane = np.array([
    [1,1,1,1,1,1,1, 0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0, 1,1,1,1,1,1,1]
])

S_steane = np.array([
        # g1: IIIXXX
        [1,1,1,1,0,0,0, 0,0,0,0,0,0,0],
        # g2: IXXIIXX  
        [0,1,1,0,1,1,0, 0,0,0,0,0,0,0],
        # g3: XIXIXIX
        [0,0,1,1,0,1,1, 0,0,0,0,0,0,0],
        # g4: IIIZZZ
        [0,0,0,0,0,0,0, 1,1,1,1,0,0,0],
        # g5: IZZIIZZ
        [0,0,0,0,0,0,0, 0,1,1,0,1,1,0],
        # g6: ZIZIZI
        [0,0,0,0,0,0,0, 0,0,1,1,0,1,1]
    ])

In [40]:
distGenetic(L_steane, S=S_steane, tB=tB, pList=plist, settgs=settings)

(np.float64(7.3778221926e-11),
 array([[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0]]))

## small Graph code case

Consider the liner graph state
```mermaid
graph LR
    A((1)) --- B((2)) --- C((3))
```

This graph has stabilizer 

| Generator | Node 1 | Node 2 | Node 3 |
|-----------|--------|--------|--------|
| S1        | X      | Z      | Z      |
| S2        | Z      | X      | Z      |
| S3        | Z      | Z      | X      |

This graph can be seen the code

| Generator | Node 1 | Node 2 |
|-----------|--------|--------|
| S1        | Y      | Y      |
| LX        | Z      | Z      |
| S3        | X      | Z      |

Let evaluate it's error rate


In [28]:
L_graph = np.array([
    [0,0, 1,1],
    [1,0, 0,1]
])

In [29]:
S_graph = np.array([
    [1,1, 1,1],
])

In [30]:
probList, LOList, population = distGeneticEvalInterface(L_graph, S=S_graph, tB=tB, pList=plist, settgs=settings)

print(
    "probList",
    probList
)
print(
    "LOList",
    LOList
)
print(
    "population",
    population
)

probList [np.float64(0.001)]
LOList [array([[0, 0, 1, 1, 1, 0],
       [1, 0, 1, 0, 1, 1]])]
population [array([1, 0])]


In [26]:
distGenetic(L, S=S, tB=tB, pList=plist, settgs=settings)

(np.float64(0.001),
 array([[0, 0, 1, 1],
        [0, 1, 0, 1]]))